In [14]:
from valurap2 import commands
from valurap2 import buf_commands
import time
import imp
imp.reload(buf_commands)
imp.reload(commands)

p = commands.S3GPort()
p.S3G_OUTPUT(63, 33)
v = p.S3G_INPUT(63)
display(v)

cb = buf_commands.CommandBuffer()
cb.hw_reset()
cb.BUF_DONE()

a = p.S3G_INPUT(cb.IN_MOTOR1_X)
print("X1:", a)
a = p.S3G_INPUT(cb.IN_MOTOR2_X)
print("X2:", a)

p.S3G_STB(cb.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
while True:
    a = p.S3G_INPUT(cb.IN_APG_STATUS)
    print("APG_STATUS", a)
    a = p.S3G_INPUT(cb.IN_BE_STATUS)
    print("BE_STATUS", a)
    if cb.extract_field(cb.IN_BE_STATUS_BUSY, a) == 0:
        break
    time.sleep(0.1)

a = p.S3G_INPUT(cb.IN_MOTOR1_X)
print("X1:", a)
a = p.S3G_INPUT(cb.IN_MOTOR2_X)
print("X2:", a)


33

X1: 80000
X2: 80000
APG_STATUS 1
BE_STATUS 0
X1: 0
X2: 0


In [2]:

s_t = 0.001
f = 50000000
sb = 40

n = 400 * 32
t = 1

v = n * (2 ** sb) / f / t
v1 = v * 2
n_s = t / s_t
a = v1 / n_s
dt = f * s_t
v, a, dt, n_s

(281474976.710656, 562949.9534213119, 50000.0, 1000.0)

In [16]:
import time
from valurap2.buf_commands import CommandBuffer
cb = CommandBuffer(debug=True)

cb.BUF_OUTPUT(0, 50000)        # step time 0.001s
cb.BUF_OUTPUT(2, 40)           # step_bit = 40
cb.BUF_OUTPUT(3, 100)          # pre_n
cb.BUF_OUTPUT(4, 400)          # pulse_n
cb.BUF_OUTPUT(5, 500)          # post_n
cb.BUF_OUTPUT(7, 0x80818080)   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(8, 0x80818080)   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(9, 0x80818080)   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(13, 500000)       # ES_TIMEOUT 10ms

cb.BUF_OUTPUT(1, 5000)              # steps_val = 2000 (2 seconds)
cb.write_param(0, cb.PARAM_STATUS, cb.PARAM_STATUS_ENABLE | cb.PARAM_STATUS_TARGET_V)           # CH0.STATUS=3 (enable + target_v)
cb.write_param(0, cb.PARAM_V_OUT, 0)
cb.write_param(0, cb.PARAM_A , 500000)
cb.write_param(0, cb.PARAM_J, 0)
cb.write_param(0, cb.PARAM_JJ, 0)
cb.write_param(0, cb.PARAM_TARGET_V, 50000000)
cb.write_param(0, cb.PARAM_ABORT_A, 500000)
for ch in range(1, 8):
    cb.write_param(ch, 0, 0)

cb.BUF_STB(cb.STB_ASG_START)   # Start ASG cycles
cb.BUF_WAIT_ALL(cb.INT_ASG_LOAD)  # Wait for APG ready for new segment
cb.BUF_CLEAR(cb.INT_ASG_LOAD)     # Clear pending int

cb.BUF_OUTPUT(1, 3000)           # steps_val = 3000 (3 seconds)
cb.write_param(0, cb.PARAM_STATUS, cb.PARAM_STATUS_ENABLE | cb.PARAM_STATUS_TARGET_V)           # CH0.STATUS=3 (enable + target_v)
cb.write_param(0, cb.PARAM_A , -500000)
cb.write_param(0, cb.PARAM_TARGET_V, 5000000)

cb.BUF_STB(cb.STB_ASG_LOAD_DONE)  # Inform ASG, next segment setup is done
cb.BUF_WAIT_ALL(cb.INT_ASG_LOAD)
cb.BUF_CLEAR(cb.INT_ASG_LOAD)

cb.BUF_OUTPUT(1, 1000)      # steps_val = 1000 (1 second)

cb.write_param(0, cb.PARAM_STATUS, cb.PARAM_STATUS_ENABLE | cb.PARAM_STATUS_TARGET_V)           # CH0.STATUS=3 (enable + target_v)
cb.write_param(0, cb.PARAM_A , -500000)
cb.write_param(0, cb.PARAM_TARGET_V, 0)

cb.BUF_STB(cb.STB_ASG_LOAD_DONE)
cb.BUF_WAIT_ALL(cb.INT_ASG_LOAD)
cb.BUF_CLEAR(cb.INT_ASG_LOAD)

cb.BUF_OUTPUT(1, 0)         # steps_val = 0 - End of path
cb.BUF_STB(cb.STB_ASG_LOAD_DONE)
cb.BUF_WAIT_ALL(cb.INT_ASG_DONE)  # Wait for ASG done
cb.BUF_CLEAR(cb.INT_ASG_DONE)

cb.BUF_OUTPUT(7, 0x00000000) # disable 1 and 2 motors
cb.BUF_OUTPUT(8, 0x00000000) # disable 1 and 2 motors
cb.BUF_OUTPUT(9, 0x00000000) # disable 1 and 2 motors
cb.BUF_DONE(0)                # End of program for BE

p.S3G_STB(cb.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_STB(cb.STB_ASG_ABORT)
p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
p.S3G_STB(cb.STB_ES_UNLOCK)

oa, ob, oc, od1, od2 = None, None, None, -1, -1
pd1 = -1
pd2 = -1
while True:
    a = p.S3G_INPUT(cb.IN_BE_STATUS)
    b = p.S3G_INPUT(cb.IN_APG_STATUS)
    c = p.S3G_INPUT(cb.IN_FIFO_DATA_COUNT)
    d1 = p.S3G_INPUT(cb.IN_MOTOR1_HOLD_X)
    d2 = p.S3G_INPUT(cb.IN_MOTOR2_HOLD_X)
    
    if (a, b, c, d1, d2) != (oa, ob, oc, od1, od2):
        print(a,b,c,d1,d2, od1 - d1, od2 - d2)
        if (d1, d2) != (od1, od2):
            es = p.S3G_INPUT(cb.IN_ES_STATUS)
            if es & 2:
                print("es1", es, d1 - pd1)
                pd1 = d1
                p.S3G_STB(cb.STB_ES_UNLOCK)
            if es & 32:
                print("es1", es, d2 - pd2)
                pd2 = d2
                p.S3G_STB(cb.STB_ES_UNLOCK)
        
    (oa, ob, oc, od1, od2) = (a,b,c,d1,d2) 
    if a & 1 == 0:
        break
    time.sleep(0.001)

BUF_OUTPUT(0, 50000)
BUF_OUTPUT(2, 40)
BUF_OUTPUT(3, 100)
BUF_OUTPUT(4, 400)
BUF_OUTPUT(5, 500)
BUF_OUTPUT(7, -2138996608)
BUF_OUTPUT(8, -2138996608)
BUF_OUTPUT(9, -2138996608)
BUF_OUTPUT(13, 500000)
BUF_OUTPUT(1, 5000)
BUF_PARAM_ADDR(0)
BUF_PARAM_WRITE_LO(0, 3)
BUF_PARAM_WRITE_LO(3, 0)
BUF_PARAM_WRITE_LO(1, 500000)
BUF_PARAM_WRITE_LO(1, 0)
BUF_PARAM_WRITE_LO(1, 0)
BUF_PARAM_WRITE_LO(1, 50000000)
BUF_PARAM_WRITE_LO(1, 500000)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_PARAM_WRITE_LO_NC(0)
BUF_STB(0x00000004)
BUF_WAIT_ALL(0x00000002)
BUF_CLEAR(0x00000002)
BUF_OUTPUT(1, 3000)
BUF_PARAM_WRITE_LO_NC(3)
BUF_PARAM_WRITE_LO(4, -500000)
BUF_PARAM_WRITE_LO(3, 5000000)
BUF_STB(0x00000010)
BUF_WAIT_ALL(0x00000002)
BUF_CLEAR(0x00000002)
BUF_OUTPUT(1, 1000)
BUF_PARAM_ADDR(0)
BUF_PARAM_WRITE_LO(0, 3)
BUF_PARAM_WRITE_LO(4, -500000)
BUF_PARAM_WRITE_LO(3, 0)
BUF_STB(0x00000010)
BUF_WAIT_ALL(0x

In [109]:
oa = None
p.S3G_STB(cb.STB_ES_UNLOCK)

while True:
    a = p.S3G_INPUT(cb.IN_ES_STATUS)
    
    if a != oa:
        print(a)
    
    p.S3G_STB(cb.STB_ES_UNLOCK)
        
    oa = a 
    time.sleep(0.01)

0
3
1
2
0
3
1
2
0
3
0
1
2
0
3
1
2
0
48
0
48
0
48
0
48
0
48
16
32
48
0
48
16
32
0
48
16
32
0
48
16
32
0
48
16
32
0
48
16
32
0
768
256
512
0
768
256
512
0
768
256
512
0
12288
4096
0
12288
4096
8192
0
196608
65536
131072
0
196608
65536
131072
0
196608
65536
131072
196608
0
196608
65536
131072
0
196608
65536
131072
0
3145728
1048576
2097152
0
3145728
2097152
1048576
2097152
0
3145728
0
3145728
1048576
2097152
0
3145728
1048576
2097152
3145728
0
50331648
0
50331648
0
50331648
16777216
33554432
0
50331648
16777216
33554432
0
50331648
16777216
33554432
0
50331648
16777216
33554432
0
50331648
16777216
33554432
0
805306368
268435456
536870912
0
805306368
268435456
536870912
0
805306368
268435456
536870912
0
268435456
0
805306368
268435456
536870912
0
805306368
268435456
536870912
805306368
0


KeyboardInterrupt: 

In [14]:
from valurap2.buf_commands import CommandBuffer
import time

cb = CommandBuffer()

cb.BUF_OUTPUT(0, 50000),        # step time 0.001s
cb.BUF_OUTPUT(2, 40),           # step_bit = 40
cb.BUF_OUTPUT(3, 100),          # pre_n
cb.BUF_OUTPUT(4, 400),          # pulse_n
cb.BUF_OUTPUT(5, 500),          # post_n
cb.BUF_OUTPUT(7, 0x80988089),   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(13, 500000),       # ES_TIMEOUT 10ms

cb.BUF_OUTPUT(1, 20000),              # steps_val = 20000 (20 seconds)
cb.BUF_PARAM_ADDR(0),
cb.BUF_PARAM_WRITE_LO(0, 3),           # CH0.STATUS=3 (enable + target_v)
cb.BUF_PARAM_WRITE_LO(3, 0),           # CH0.V_OUT=0
cb.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.A
cb.BUF_PARAM_WRITE_LO(1, 0),           # CH0.J=0
cb.BUF_PARAM_WRITE_LO(1, 0),           # CH0.JJ=0
cb.BUF_PARAM_WRITE_LO(1, 210000000),  # CH0.TARGET_V
cb.BUF_PARAM_WRITE_LO(1, 15000000),     # CH0.ABORT_A

cb.BUF_PARAM_WRITE_LO_NC(3),           # CH0.STATUS=3 (enable + target_v)
cb.BUF_PARAM_WRITE_LO(3, 0),           # CH0.V_OUT=0
cb.BUF_PARAM_WRITE_LO(1, 500000),     # CH0.A
cb.BUF_PARAM_WRITE_LO(1, 0),           # CH0.J=0
cb.BUF_PARAM_WRITE_LO(1, 0),           # CH0.JJ=0
cb.BUF_PARAM_WRITE_LO(1, 70000000),  # CH0.TARGET_V
cb.BUF_PARAM_WRITE_LO(1, 15000000),     # CH0.ABORT_A

cb.BUF_PARAM_WRITE_LO_NC(0),   # CH2
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH3
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH4
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH5
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH6
cb.BUF_PARAM_WRITE_LO_NC(0),   # CH7
cb.BUF_STB(cb.STB_ASG_START),   # Start ASG cycles

cb.BUF_DONE(0)                # End of program for BE

p.S3G_STB(cb.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_STB(cb.STB_ASG_ABORT)
p.S3G_STB(cb.STB_ES_UNLOCK)
p.S3G_STB(cb.STB_SP_ZERO)
p.S3G_CLEAR(0xffffffff)
p.S3G_OUTPUT(cb.OUT_MSG_X_VAL, 0)
p.S3G_OUTPUT(cb.OUT_MSG_CONFIG0, 0x82008200)
p.S3G_STB(cb.STB_MSG_SET_X)
p.S3G_OUTPUT(cb.OUT_MSG_CONFIG0, 0x80008000)
p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution

oa, ob, oc, od1, od2 = None, None, None, -1, -1
pd1 = -1
pd2 = -1
oes = None
while True:
    a = p.S3G_INPUT(cb.IN_BE_STATUS)
    b = p.S3G_INPUT(cb.IN_APG_STATUS)
    c = p.S3G_INPUT(cb.IN_FIFO_DATA_COUNT)
    d1 = p.S3G_INPUT(cb.IN_MOTOR1_HOLD_X)
    d2 = p.S3G_INPUT(cb.IN_MOTOR2_HOLD_X)
    ints = p.S3G_INPUT(cb.IN_PENDING_INTS)
    es = p.S3G_INPUT(cb.IN_ES_STATUS)
    
    if (a, b, c, d1, d2, es) != (oa, ob, oc, od1, od2, oes):
        print(a,b,c,d1, d2, od1 - d1, od2 - d2, "{:b}".format(ints))
        if (d1, d2) != (od1, od2):
            if es & 32:
                print("es1", es, d1 - pd1)
                pd1 = d1
                p.S3G_STB(cb.STB_ES_UNLOCK)
            if es & 2:
                print("es2", es, d2 - pd2)
                pd2 = d2
                p.S3G_STB(cb.STB_ES_UNLOCK)
    if ints & (cb.INT_ASG_DONE):
        print("stops are not reached")
        break
    if (ints & (cb.INT_APG0_ABORT_DONE)) and (ints & (cb.INT_APG1_ABORT_DONE)):
        print("Both stops are reached")
        cd1 = p.S3G_INPUT(cb.IN_MOTOR1_X)
        cd2 = p.S3G_INPUT(cb.IN_MOTOR2_X)
        print(pd1, cd1, cd1 - pd1, pd2, cd2, cd2 - pd2)
        break
        
    (oa, ob, oc, od1, od2, oes) = (a,b,c,d1,d2,es) 
    #if a & 1 == 0:
    #    break
    time.sleep(0.001)

0 1 0 17 67 -18 -68 1
0 1 0 3088 67 -3071 0 100000001
es1 48 3089
0 1 0 3088 67 0 0 100000001
0 1 0 3088 67 0 0 100000001
0 1 0 3088 3112 0 -3045 1100000001
es2 19 3113
Both stops are reached
3088 3180 92 3112 3127 15


In [46]:
from valurap2 import emulate
from valurap2 import commands
from valurap2 import buf_commands

import pickle
import imp

imp.reload(buf_commands)
imp.reload(commands)

cmds = pickle.load(open("commands.pck", "rb"))

cb = buf_commands.CommandBuffer()
#cb.hw_reset()
cb.BUF_OUTPUT(0, 50000)        # step time 0.001s
cb.BUF_OUTPUT(2, 40)           # step_bit = 40
cb.BUF_OUTPUT(3, 100)          # pre_n
cb.BUF_OUTPUT(4, 400)          # pulse_n
cb.BUF_OUTPUT(5, 500)          # post_n
cb.BUF_OUTPUT(7, 0x80a08090)   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(8, 0x800080d0)   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(9, 0x80008000)   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(13, 500000)       # ES_TIMEOUT 10ms
cb.buffer.extend(cmds)
cb.BUF_OUTPUT(7, 0x00000000)   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(8, 0x00000000)   # enable 1 and 2 motors on SG0
cb.BUF_OUTPUT(9, 0x00000000)   # enable 1 and 2 motors on SG0
cb.BUF_DONE()

p = commands.S3GPort()
p.S3G_STB(cb.STB_BE_ABORT)      # Reset BE and ASG just in case
p.S3G_STB(cb.STB_ASG_ABORT)
p.S3G_STB(cb.STB_ES_UNLOCK)
p.S3G_STB(cb.STB_SP_ZERO)
p.S3G_CLEAR(0xffffffff)
free_space, status = p.S3G_WRITE_FIFO(cb, timeout=0.1, until_free=1000)           # Send program into FIFO
print(free_space, status)
p.S3G_STB(cb.STB_BE_START)      # Start execution
free_space, status = p.S3G_WRITE_FIFO(cb, timeout=0.1, max_cmds=100)           # Send program into FIFO
print(free_space, status)
while True:
    if cb.buffer:
        free_space, status = p.S3G_WRITE_FIFO(cb, until_free=1000)           # Send program into FIFO
        if status & 1 == 0:
            break
        print(free_space, status)
    else:
        c = p.S3G_INPUT(cb.IN_FIFO_DATA_COUNT)
        a = p.S3G_INPUT(cb.IN_BE_STATUS)
        print("STATUS", a, c)
        if cb.extract_field(cb.IN_BE_STATUS_BUSY, a) == 0:
            break
    time.sleep(0.1)



992 0
943 1
903 1
863 1
823 1
783 1
762 1
722 1
758 1
718 1
716 1
714 1
712 1
710 1
727 1
725 1
742 1
759 1
776 1
774 1
772 1
789 1
806 1
861 1
897 1
971 1
950 1
967 1
982 1
961 1
978 1
974 1
991 1
989 1
983 1
981 1
996 1
971 1
967 1
980 1
976 1
993 1
989 1
985 1
964 1
981 1
975 1
973 1
990 1
965 1
925 1
904 1
921 1
881 1
917 1
877 1
932 1
949 1
964 1
943 1
998 1
973 1
961 1
976 1
989 1
985 1
983 1
960 1
996 1
992 1
990 1
967 1
962 1
979 1
994 1
988 1
965 1
961 1
976 1
972 1
970 1
987 1
983 1
960 1
958 1
994 1
971 1
967 1
927 1
963 1
942 1
959 1
957 1
996 1
975 1
971 1
986 1
982 1
995 1
974 1
991 1
970 1
966 1
964 1
961 1
999 1
977 1
996 1
992 1
988 1
986 1
984 1
980 1
940 1
976 1
936 1
953 1
988 1
999 1
974 1
970 1
985 1
981 1
977 1
992 1
988 1
984 1
999 1
995 1
974 1
970 1
964 1
960 1
977 1
992 1
988 1
982 1
995 1
968 1
997 1
972 1
970 1
966 1
962 1
998 1
994 1
992 1
971 1
988 1
984 1
997 1
972 1
989 1
988 1
987 1
983 1
983 1
999 1
998 1
997 1
977 1
974 1
954 1
971 1
984 1
980 1
997 

In [20]:
from valurap2 import path_planning
from valurap2 import buf_commands, commands, spi
import time
import imp
imp.reload(buf_commands)
imp.reload(path_planning)
imp.reload(commands)

spi_port = spi.SPIPort()
spi_port.setup_tmc2130()

p = commands.S3GPort()

pp = path_planning.PathPlanner()
pp.emu_in_loop = True
pp.delta_e_err = 10
pp.delta_ve_err = 10
pp.max_ea = 10000
pp.accel_step = 5000
pp.spms = pp.home_spms

pp.init_apgs()

def wait_done():
    while True:
        #a = p.S3G_INPUT(cb.IN_APG_STATUS)
        #print("APG_STATUS", a)
        a = p.S3G_INPUT(cb.IN_BE_STATUS)
        #print("BE_STATUS", a)
        if cb.extract_field(cb.IN_BE_STATUS_BUSY, a) == 0:
            break
        time.sleep(0.1)

def wait_stops(stops=[]):
    apg0 = cb.INT_APG0_ABORT_DONE
    mask = 0
    for stop in stops:
        mask |= apg0 << stop
        
    last_stops = 0
        
    while True:
        #a = p.S3G_INPUT(cb.IN_APG_STATUS)
        #print("APG_STATUS", a)
        b = p.S3G_INPUT(cb.IN_BE_STATUS)
        #print("BE_STATUS", b)
        ints = p.S3G_INPUT(cb.IN_PENDING_INTS)
        if cb.extract_field(cb.IN_BE_STATUS_BUSY, b) == 0:
            print("Stops not reached: {:X} != {:X}".format(ints & mask, mask))
            return False
        if ints & mask != last_stops:
            print("stops done: {:X} expected: {:X}".format(ints&mask, mask))
            last_stops = ints & mask
            
        if ints & mask == mask:
            print("All stops done")
            return True
        #print("ints: {:X}".format(ints))
        time.sleep(0.1)
        
# Reset
cb = buf_commands.CommandBuffer()
p.S3G_STB(cb.STB_BE_ABORT)      # Reset BE and ASG just in case
cb.hw_reset()
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_done()

# Home configuration
# SG:   X1  X2  YL  YR  ZFL ZFR ZBL ZBR
#        0   1   2   3   4   5   6   7
#
# Print configuration
# SG:   X1  X2   Y   Z  E1  E2  --  --
#        0   1   2   3   4   5   6   7
#
# EndStops: X2  X1  YL  YR  ZBR ZBL ZFR ZFL
#            0   1   2   3   4   5   6   7
#
# Motor  Axe  ES Invert HomeSG PrintSG HomeCFG PrintCFG
#   1    ZFL   7    -      4     3      80cf     80b0
#   2    ZBR   4    -      7     3      80fc     80b0
#   3    ZBL   5    -      6     3      80ed     80b0
#   4    E1    -    -      -     4      0000     80c0
#   5    E2    -    -      -     5      0000     80d0
#   6    ZFR   6    -      5     3      80de     80b0
#   7    X1    1    I      0     0      8089     8080
#   8    X2    0    -      1     1      8198     8190
#   9    --                             0000     0000
#  10    --                             0000     0000
#  11    YR    3    -      3     2      81bb     81a0
#  12    YL    2    I      2     2      80aa     80a0

# Move Z down
cb = buf_commands.CommandBuffer()
cb.enable_axes(["home"])
cb.add_segments_head(pp)
cb.add_segments(pp.ext_to_code(20, speed=10, axes=["ZFR","ZFL","ZBR","ZBL"]))
cb.add_segments_tail(pp)
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_done()

p.S3G_STB(cb.STB_ES_UNLOCK)
time.sleep(0.1)
p.S3G_STB(cb.STB_ES_UNLOCK)
es = p.S3G_INPUT(cb.IN_ES_STATUS)
print("es_status: {:X}".format(es))
on_stops = []
if es & 0x1:
    on_stops.append("X2")
if es & 0x10:
    on_stops.append("-X1")
if es & 0x1100:
    on_stops.append("YL")
    on_stops.append("YR")

if on_stops:
    # Move XY from stops
    cb = buf_commands.CommandBuffer()
    cb.enable_axes(["home"])
    cb.add_segments_head(pp)
    cb.add_segments(pp.ext_to_code(10, axes=on_stops, speed=10))
    cb.add_segments_tail(pp)
    cb.BUF_DONE()

    p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
    p.S3G_STB(cb.STB_BE_START)      # Start execution
    wait_done()
    
# First Home XY
cb = buf_commands.CommandBuffer()
cb.enable_axes(["home", "es_xy"])
cb.add_segments_head(pp)
cb.add_segments(pp.ext_to_code(-1000, axes=["-X1","X2","YL","YR"], speed=50))
cb.add_segments_tail(pp)
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_stops([0,1,2,3])
p.S3G_STB(cb.STB_BE_ABORT)
p.S3G_STB(cb.STB_ASG_ABORT)

# Move XY from stops
cb = buf_commands.CommandBuffer()
cb.enable_axes(["home"])
cb.add_segments_head(pp)
cb.add_segments(pp.ext_to_code(30, axes=["-X1","X2","YL","YR"], speed=10))
cb.add_segments_tail(pp)
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_done()

# Second Home XY
cb = buf_commands.CommandBuffer()
cb.enable_axes(["home", "es_xy"])
cb.add_segments_head(pp)
cb.add_segments(pp.ext_to_code(-300, axes=["-X1","X2","YL","YR"], speed=10))
cb.add_segments_tail(pp)
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_stops([0,1,2,3])
p.S3G_STB(cb.STB_BE_ABORT)
p.S3G_STB(cb.STB_ASG_ABORT)

# Home Z
cb = buf_commands.CommandBuffer()
cb.enable_axes(["home", "es_z"])
cb.add_segments_head(pp)
cb.add_segments(pp.ext_to_code(-700, axes=["ZFR","ZFL","ZBR","ZBL"], speed=10))
cb.add_segments_tail(pp)
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_stops([4,5,6,7])
p.S3G_STB(cb.STB_BE_ABORT)
p.S3G_STB(cb.STB_ASG_ABORT)
   
    
# Move Z from stops
cb = buf_commands.CommandBuffer()
cb.enable_axes(["home"])
cb.add_segments_head(pp)
cb.add_segments(pp.ext_to_code(5, axes=["ZFR","ZFL","ZBR","ZBL"], speed=2))
cb.add_segments_tail(pp)
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_done()

# Second Home Z
cb = buf_commands.CommandBuffer()
cb.enable_axes(["home", "es_z"])
cb.add_segments_head(pp)
cb.add_segments(pp.ext_to_code(-20, axes=["ZFR","ZFL","ZBR","ZBL"], speed=2))
cb.add_segments_tail(pp)
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_stops([4,5,6,7])
p.S3G_STB(cb.STB_BE_ABORT)
p.S3G_STB(cb.STB_ASG_ABORT)

print(p.S3G_INPUT(cb.IN_MOTOR1_X))
print(p.S3G_INPUT(cb.IN_MOTOR1_HOLD_X))
print(p.S3G_INPUT(cb.IN_MOTOR2_X))
print(p.S3G_INPUT(cb.IN_MOTOR2_HOLD_X))
print(p.S3G_INPUT(cb.IN_MOTOR3_X))
print(p.S3G_INPUT(cb.IN_MOTOR3_HOLD_X))
print(p.S3G_INPUT(cb.IN_MOTOR6_X))
print(p.S3G_INPUT(cb.IN_MOTOR6_HOLD_X))

cb.reset()
cb.enable_axes([])
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_done()


delta_x: 6 -4.4881801386509323e-07 0.000625
delta_x: 7 -4.4881801386509323e-07 0.000625
delta_x: 4 -4.4881801386509323e-07 0.000625
delta_x: 5 -4.4881801386509323e-07 0.000625


TypeError: add_segments_tail() takes 1 positional argument but 2 were given

In [50]:
p.S3G_OUTPUT(7, 0x00000000)   # enable 1 and 2 motors on SG0
p.S3G_OUTPUT(8, 0x00000000)   # enable 1 and 2 motors on SG0
p.S3G_OUTPUT(9, 0x00000000)   # enable 1 and 2 motors on SG0
p.S3G_OUTPUT(10, 0x00000000)   # enable 1 and 2 motors on SG0
p.S3G_OUTPUT(11, 0x00000000)   # enable 1 and 2 motors on SG0
p.S3G_OUTPUT(12, 0x00000000)   # enable 1 and 2 motors on SG0


In [32]:
p.S3G_STB(cb.STB_ES_UNLOCK)
p.S3G_STB(cb.STB_ES_UNLOCK)
es = p.S3G_INPUT(cb.IN_ES_STATUS)
print("es_status: {:X}".format(es))

es_status: 1000


In [12]:
# Move Z down
cb = buf_commands.CommandBuffer()
cb.enable_axes(["home"])
cb.add_segments_head(pp)
cb.add_segments(pp.ext_to_code(20, speed=10, axes=["ZFR","ZFL","ZBR","ZBL"]))
cb.add_segments_tail(pp)
cb.BUF_DONE()

p.S3G_WRITE_FIFO(cb)           # Send program into FIFO
p.S3G_STB(cb.STB_BE_START)      # Start execution
wait_done()

delta_x: 6 -4.4881801386509323e-07 0.000625
delta_x: 7 -4.4881801386509323e-07 0.000625
delta_x: 4 -4.4881801386509323e-07 0.000625
delta_x: 5 -4.4881801386509323e-07 0.000625


TypeError: add_segments_tail() takes 1 positional argument but 2 were given

In [8]:
from valurap2 import printer
import imp
imp.reload(printer)

prn = printer.Valurap()
prn.setup()
prn.home()

Executing reset code
Reset OLED
Setup done
c 1 0
h 1 34405
c 2 0
h 2 29595
c 3 0
h 3 35985
c 6 0
h 6 28015
delta_x: 6 -6.691612952636206e-07 0.000625
delta_x: 7 -6.691612952636206e-07 0.000625
delta_x: 4 -6.691612952636206e-07 0.000625
delta_x: 5 -6.691612952636206e-07 0.000625
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'b

Status: wait stops {'be_status': 9, 'stops': 0, 'expected_stops': 61440}
Status: wait stops {'be_status': 9, 'stops': 0, 'expected_stops': 61440}
Status: wait stops {'be_status': 9, 'stops': 0, 'expected_stops': 61440}
Status: wait stops {'be_status': 9, 'stops': 0, 'expected_stops': 61440}
stops done: 2000 expected: F000
Status: wait stops {'be_status': 9, 'stops': 8192, 'expected_stops': 61440}
stops done: A000 expected: F000
Status: wait stops {'be_status': 9, 'stops': 40960, 'expected_stops': 61440}
stops done: F000 expected: F000
All stops done
delta_x: 6 -1.2600821719388478e-07 0.000625
delta_x: 7 -1.2600821719388478e-07 0.000625
delta_x: 4 -1.2600821719388478e-07 0.000625
delta_x: 5 -1.2600821719388478e-07 0.000625
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_

In [6]:
prn.moveto(X1=0,X2=0, modes=["print", "e1", "e2"])

xc 154.9875 dx 0
absolute dx: -154.9875
xc -170.0 dx 0
absolute dx: 170.0
segments: dxes: [-154.9875, 170.0] axes: ['X1', 'X2'] speed: 150
delta_x: 0 0.004556081937437284 0.0125
delta_x: 1 -0.004997373852717146 0.0125
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}
Status: wait done {'be_status': 9}


0